# Test MBTR predicting ground state energies of binary systems using KRR 

In [25]:
'''
Authors Brayden Bekker, Chandramouli Nyshadham

Developed as part of the MBTR project using EAM files.

Input:
- [system].eam.alloy: where system is a binary or ternary system of elements from the periodic
table. the potentials were gathered from the ctcms Interatomic Potentials Repository. 
- Structures_[system]/[fcc, bcc, hcp]: A hierarchy of directories for each each system 
containing 2500 derivative super structures developed with enumlib for fcc,bcc,hcp. The first
500 structures and then a random selection of 2000 structures from the 10-12 unit cells.

Output:
- HDF5 with lammps energyies data. See genHDF5 function for details.
'''
#compute total energies with:
import quippy
from lammpslib import LAMMPSlib
from aflow import *
import os
from os.path import isfile, join
import numpy as np
import collections
import matplotlib.pyplot as plt
#import h5py
import itertools
from ase import Atoms
from ase.optimize.precon import PreconLBFGS

In [26]:
def GetBinarySystems(arg1,arg2):
    '''
    load the EAM potential files for a system along with element data.
    os.chdir(~/codes/MBTR_local/) with eam_potentials/ containing the NIST files.

    '''


    # The dictionary of all the elements on the periodic table
    elements ={"H":3.75,"AtomicMassH": 1.0079,"AtomicNumH": 1,"He": 3.57,"AtomicMassHe":4.0026,"AtomicNumHe":2, 
               "Li":3.49,"AtomicMassLi":6.941,"AtomicNumLi":3,"Be":2.29,"AtomicMassBe":9.0122,"AtomicNumBe":4,
               "B":8.73,"AtomicMassB":10.811,"AtomicNumB":5,"C": 3.57,"AtomicMassC":12.0107,"AtomicNumC":6, 
               "N":4.039,"AtomicMassN":14.0067,"AtomicNumN":7,"O":6.83,"AtomicMassO":15.9994,"AtomicNumO":8,
               "Ne":4.43,"AtomicMassNe":20.1797,"AtomicNumNe":10,"Na":4.23,"AtomicMassNa":22.9897,"AtomicNumNa":11,
               "Mg":3.21,"AtomicMassMg":24.305,"AtomicNumMg":12,"Al":4.05,"AtomicMassAl":26.9815,"AtomicNumAl":13,
               "Si":5.43,"AtomicMassSi":28.0855,"AtomicNumSi":14,"P":7.17,"AtomicMassP":30.9738,"AtomicNumP":15,
               "S":10.47,"AtomicMassS":32.065,"AtomicNumS":16,"Cl":6.24,"AtomicMassCl":35.453,"AtomicNumCl":17,
               "Ar":5.26,"AtomicMassAr":39.948,"AtomicNumAr":18,"K":5.23,"AtomicMassK":39.0983,"AtomicNumK":19, 
               "Ca":5.58,"AtomicMassCa":40.078,"AtomicNumCa":20,"Sc":3.31,"AtomicMassSc":44.9559,"AtomicNumSc":21,
               "Ti":2.95,"AtomicMassTi":47.867,"AtomicNumTi":22,"V":3.02,"AtomicMassV":50.9415,"AtomicNumV":23, 
               "Cr":2.88,"AtomicMassCr":51.9961,"AtomicNumCr":24,"Mn":8.89,"AtomicMassMn":54.938,"AtomicNumMn":25,
               "Fe":2.87,"AtomicMassFe":55.845,"AtomicNumFe":26,"Co":2.51,"AtomicMassCo":58.9332,"AtomicNumCo":27,
               "Ni":3.52,"AtomicMassNi":58.6934,"AtomicNumNi":28,"Cu":3.61,"AtomicMassCu":63.546,"AtomicNumCu":29,
               "Zn":2.66,"AtomicMassZn":65.39,"AtomicNumZn":30,"Ga":4.51,"AtomicMassGa":69.723,"AtomicNumGa":31,
               "Ge":5.66,"AtomicMassGe":72.64,"AtomicNumGe":32,"As":4.13,"AtomicMassAs":74.9216,"AtomicNumAs":33,
               "Se":4.36,"AtomicMassSe":78.96,"AtomicNumSe":34,"Br":6.67,"AtomicMassBr":79.904,"AtomicNumBr":35, 
               "Kr":5.72,"AtomicMassKr":83.8,"AtomicNumKr":36,"Rb":5.59,"AtomicMassRb":85.4678,"AtomicNumRb":37, 
               "Sr":6.08,"AtomicMassSr":87.62,"AtomicNumSr":38,"Y":3.65,"AtomicMassY":88.9059,"AtomicNumY":39, 
               "Zr":3.23,"AtomicMassZr":91.224,"AtomicNumZr":40,"Nb":3.3,"AtomicMassNb":92.9064,"AtomicNumNb":41,
               "Mo":3.15,"AtomicMassMo":95.94,"AtomicNumMo":42,"Tc":2.74,"AtomicMassTc":98,"AtomicNumTc":43, 
               "Ru":2.7,"AtomicMassRu":101.07,"AtomicNumRu":44,"Rh":3.8,"AtomicMassRh":102.9055,"AtomicNumRh":45,
               "Pd":3.89,"AtomicMassPd":106.42,"AtomicNumPd":46,"Ag":4.09,"AtomicMassAg":107.8682,"AtomicNumAg":47, 
               "Cd":2.98,"AtomicMassCd":112.411,"AtomicNumCd":48,"In":4.59,"AtomicMassIn":114.818,"AtomicNumIn":49, 
               "Sn":5.82,"AtomicMassSn":118.71,"AtomicNumSn":50,"Sb":4.51,"AtomicMassSb":121.76,"AtomicNumSb":51,
               "Te":4.45,"AtomicMassTe":127.6,"AtomicNumTe":52,"I":7.27,"AtomicMassI":126.9045,"AtomicNumI":53, 
               "Xe":6.2,"AtomicMassXe":131.293,"AtomicNumXe":54,"Cs":6.05,"AtomicMassCs":132.9055,"AtomicNumCs":55,
               "Ba":5.02,"AtomicMassBa":137.327,"AtomicNumBa":56,"Hf":3.2,"AtomicMassHf":178.49,"AtomicNumHf":72,
               "Ta":3.31,"AtomicMassTa":180.9479,"AtomicNumTa":73,"W":3.16,"AtomicMassW":183.84,"AtomicNumW":74,
               "Re":2.76,"AtomicMassRe":186.207,"AtomicNumRe":75,"Os":2.64,"AtomicMassOs":190.23,"AtomicNumOs":76,
               "Ir":3.84,"AtomicMassIr":192.217,"AtomicNumIr":77,"Pt":3.92,"AtomicMassPt":195.078,"AtomicNumPt":78,
               "Au":4.08,"AtomicMassAu":196.9665,"AtomicNumAu":79,"Hg":2.99,"AtomicMassHg":200.59,"AtomicNumHg":80,
               "Tl":3.46,"AtomicMassTl":204.3833,"AtomicNumTl":81,"Pb":4.95,"AtomicMassPb":207.2,"AtomicNumPb":82,
               "Bi":4.75,"AtomicMassBi":208.9804,"AtomicNumBi":83,"Sm":3.62,"AtomicMassSm":150.36,"AtomicNumSm":62} 

    element1="%s"%(arg1)
    element2="%s"%(arg2)
    mass1="AtomicMass%s"%(arg1)
    mass2="AtomicMass%s"%(arg2)
    number1="AtomicNum%s"%(arg1)
    number2="AtomicNum%s"%(arg2)
    system = [element1,element2,
              elements[mass1],elements[mass2],
              elements[number1],elements[number2]]
    return system

In [ ]:
def readAllVASPFiles(paths,potential,AtomicNum=[13,22]):
    """
    This fuction reads in all the crystal structures in the path to folder given. All the input files should be in
    vasp format. 
    
    The function returns atom numbers, positions(cartesian coordinates), and total energies computed using the
    potential.
    
    Arguments:
        Input: 
            path (string): path to the folder where all strucutres (vasp files) are present
            numStructToRead (int): if FALSE all files in the folder are read,else only 'numStructToRead' are read.
            potential: potential for computing the total energies.
            AtomicNum: Array containing elements atomic numbers.
            
        output:
            Returns 
            
            z (list): atomic nuclear charge
            pos (list): positions  of all atoms in a crystal structure. (cartesian) 
            lattice (list): lattice vectors of the unit cell. 
            totEnePerAtom (list): total energy per atom of crystal structure computed using potential.
            structInfo (list): contains the structure information (file number).
            conc : concentration of element A in the binary AB (listed in alphabetical order) 
            
    Note: All files in the folder should be named in the format, 'vasp.1', 'vasp.3', ... etc. which is what enumlib 
            generates.
    """
    
    AtomicNumA=AtomicNum[0]
    AtomicNumB=AtomicNum[1]

    res_z=np.array([])
    res_pos=np.array([])
    res_lattice=np.array([])
    res_totEnePerAtom=np.array([])
    res_structInfo=np.array([])
    res_conc=np.array([])

    res={}

    atomsList=quippy.AtomsList()

    for path in paths:

        for i,f  in enumerate(os.listdir(path)):
            inpFile=join(path,f) # get the file name
            a = quippy.Atoms(inpFile, format="vasp") # read the input file as quippy object 
            atomsList.append(a)
            a.set_calculator(potential) # set the potential 
            
            dyn = PreconLBFGS(a, trajectory='output.traj')
            dyn.run(fmax=1e-5, )

            res_z=np.append(res_z, list(a.z)) # get nuclear charge
            res_pos=np.append(res_pos, a.get_positions()) # get positions in cartesian
            res_lattice=np.append(res_lattice, a.get_cell())  # get lattice vectors

            totalNumOfAtoms=len(a.z) # total number of atoms
            a.params["energy"] = a.get_total_energy()
            res_totEnePerAtom=np.append(res_totEnePerAtom, a.energy/float(len(a.z))) # compute total energy per atom

            res_forces=np.append(res_forces, a.get_total_energy())
            conc=a.z.tolist().count(AtomicNumB)/float(len(a.z)) # get the concentration.
            res_conc=np.append(res_conc, conc)
            

            res_structInfo=np.append(res_structInfo, path[-4::]+str(f)) # structure information a.k.a file name.

    res={'z':res_z,'pos':res_pos,'forces':res_forces, 'lattice':res_lattice,'totEnePerAtom':res_totEnePerAtom,'structInfo':res_structInfo,'conc':res_conc}

    return res #,atomsList

def computeEnthalpy(data):
    """
    
    Input:
    data : Dictionary generated using 
    readAllVASPFiles(paths,potential,numStructToRead=0,AtomicNum=[29,73]) function.'
    
    Output:
    enthalpy added to dictionary 'data'.
    
    """
    
    res_enthalpy=np.array([])
    #find the index of locations where concentration == 0.0;
    index=[i for i,x in enumerate(data['conc']) if x == 0.0] 
    g = np.array([np.where(data['totEnePerAtom'] == np.array(data['totEnePerAtom'])[index].min())])
    print g
    fileNum=np.matrix.item(g)
    ene_A=data['totEnePerAtom'][fileNum]
    print "element A", data['structInfo'][fileNum], data['totEnePerAtom'][fileNum]

    index=[i for i,x in enumerate(data['conc']) if x == 1.0]
    g = np.array([np.where(data['totEnePerAtom'] == np.array(data['totEnePerAtom'])[index].min())])
    fileNum=np.matrix.item(g)
    ene_B=data['totEnePerAtom'][fileNum] 
    print "element B", data['structInfo'][fileNum], data['totEnePerAtom'][fileNum]

    for i in range(len(data['conc'])):
        enthalpy = data['totEnePerAtom'][i] - (1-data['conc'][i])*float(ene_A)  - (data['conc'][i])*ene_B
        res_enthalpy=np.append(res_enthalpy, enthalpy)
    
    return res_enthalpy


In [ ]:
def plotConvexHull(data,title):
    """
    Plots the convex hull.
    """
    
    fccNum=len(filter(lambda x:'fcc' in x, data['structInfo']))
    bccNum=len(filter(lambda x:'bcc' in x, data['structInfo']))
    hcpNum=len(filter(lambda x:'hcp' in x, data['structInfo']))      
    # plotting the convex hull
    plt.figure()
    plt.scatter(data['conc'][0:fccNum-1],data['enthalpy'][0:fccNum-1],label='fcc')
    plt.scatter(data['conc'][fccNum:fccNum+bccNum-1],data['enthalpy'][fccNum:fccNum+bccNum-1],label='bcc')
    plt.scatter(data['conc'][-hcpNum:],data['enthalpy'][-hcpNum:],label='hcp')
    plt.title(title)
    plt.xlim(0,1)
    plt.legend()
    plt.savefig("ConvexHull%s.pdf"%(title), format="pdf")

In [ ]:
def get_aflowData(totalStruct,listt):
    """
    Collects AFLOW data for a binary alloy containing species1 and species2
    
    """
    
    resConc=[]
    resEne=[]  
    
    global aflowListt
    
    for i in range(totalStruct):
            
        entry=listt[i]
        resConc.append(entry.stoichiometry[0])
        resEne.append(entry.enthalpy_formation_atom)
        
        
    return resConc,resEne

def plotAflowConvexHull(res1,res2,title):
    # plotting the convex hull.
    plt.figure()
    plt.scatter(res1,res2,s=10,label='alowData')
    plt.xlim(0,1)
    plt.ylim(-1,3)
    plt.legend()
    plt.title(title)
    plt.savefig("Aflow%s.pdf"%(title), format="pdf")

In [ ]:
#import h5py
#a = h5py.File("SystemEnergies1.hdf5", "w") 

In [ ]:
def getHDF5():
    '''
    create a SystemEnergies.hdf5 file containing information for MBTR:

    The following Hierarchical Structures Exists withing the file with AlTi as an example that
    is repeted for each system in the file.

    SystemEnergies/AlTi/StructInfo: String (e.g. AlTi, fcc, vasp.c)
    SystemEnergies/AlTi/Z: Int (e.g. Nuclear Charges for given vasp.c)
    SystemEnergies/AlTi/BasisVector: Int (e.g. Basis Lattice Vector for given vasp.c)
    SystemEnergies/AlTi/TotAtomEnergy: Int (e.g. Total energy per atom for given vasp.c)
    SystemEnergies/AlTi/TotAtomEnthalpy: Int (e.g. Total enthalpy per atom for given vasp.c)
    SystemEnergies/AlTi/Concentration: Int (e.g. Concentration for given vasp.c)
    '''
    numSystems = 17 #the number of systems to test
    Alsubsystem = ["Co","Ni","Pb","Ti"]
    for i in range(numSystems):
        #if(i==0): arg1, arg2, arg3 = "Al", "Cu", "eam"
        if(i==0): arg1, arg2, arg3 = "Al", "Co", "alloy"
        print arg1, arg2, arg3
        system = GetBinarySystems(arg1, arg2)

        grp = a.create_group("%s%s"%(system[0],system[1]))

        # loading the interatomic potential.
        header = ["units metal",
                  "dimension 3",
                  "boundary p p p",
                  "atom_style atomic",
                  "atom_modify map array"]
        cmds = ["pair_style eam/%s"%(arg3),
                "pair_coeff * * /root/codes/MBTR_local/eam_potentials/%s-%s.eam.%s %s %s"%(system[0],system[1], arg3, system[0],system[1]),
                "mass 1 %s"%(system[2]),
                "mass 2 %s"%(system[3]),
                "neighbor 2.0 bin",
                "neigh_modify delay 10 check yes"]
        pot = LAMMPSlib(lmpcmds=cmds, atom_types={"%s"%(system[0]): 1, "%s"%(system[1]): 2},
                        lammps_header=header, keep_alive=True)

        pathToDirectories=os.getcwd()+str('/Structures_EAM') #remove MBTR path later
        b="%s%s"%(system[0],system[1])
        folders=[pathToDirectories+str(i) for i in ['/Structures_%s/fcc/'%(b),'/Structures_%s/bcc/'%(b),'/Structures_%s/hcp/'%(b)]]

        data=readAllVASPFiles(folders,potential=pot,AtomicNum=[int(system[4]), int(system[5])])

        data.update({'enthalpy':computeEnthalpy(data)})

        plotConvexHull(data,title="%s%s"%(system[0],system[1]))

        aflowList= search().filter(K.species=="%s,%s"%(system[0],system[1])).filter(K.nspecies==2).orderby(K.enthalpy_formation_atom)

        totalStruct=len(aflowList)

        res1,res2=get_aflowData(totalStruct, aflowList)

        plotAflowConvexHull(res1,res2,"%s%s"%(system[0],system[1]))

        #structSet = grp.create_dataset("StructInfo", data=data["structInfo"])
        #zSet = grp.create_dataset("Z", data=data['z'])
        #posSet = grp.create_dataset("Position", data=data["pos"])
        #vecSet = grp.create_dataset("BasisVector", data=data["lattice"])
        #eneSet = grp.create_dataset("TotAtomEnergy", data=data["totEnePerAtom"])
        #enthSet = grp.create_dataset("TotAtomEnthalpy", data=data["enthalpy"])
        #concSet = grp.create_dataset("Concentration", data=data["conc"])
        
getHDF5()

In [ ]:
#a.close()